In [1]:
### 常规分析包
import os
import time
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm, pearsonr, spearmanr
from scipy.spatial import distance
### 绘图包
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42 # 输出PDF文字可编辑
### 其他
import cloudpickle as pickle
import warnings
warnings.filterwarnings("ignore") # 报错提示
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #last_expr

# Abundance

In [12]:
path = '../Data/WMS/02Metaphlan3/'
namemap = {'1_2018_8_20':'1_26', 'B19_5':'19_5'}
files = os.listdir(path)
abundance = pd.DataFrame()
colummns = []
for f in files:
    name = f.split('_pro')[0].strip()
    temp = pd.read_csv(path+f, sep='\t', index_col=0, skiprows=3)
    #temp
    abundance = pd.concat([abundance, temp['relative_abundance']], axis=1, sort=False)
    colummns.append('C'+namemap.get(name, name))
abundance.columns = colummns
abundance = abundance.fillna(0.0)
abundance.head(3)

,C11_13,C11_22,C11_25,C11_26,C11_27,C11_4,C11_5,C12_13,C12_22,C12_25,...,C33_5,C35_22,C35_5,C4_26,C4_27,C5_22,C5_25,C5_27,C5_4,C19_5
k__Bacteria,99.94282,99.95679,74.18245,98.62872,99.78823,99.95500,99.95906,99.99377,98.93692,99.48205,...,99.93069,92.84424,99.99640,99.39231,99.99609,99.94547,99.80360,99.83702,99.56896,99.92849
k__Viruses,0.05718,0.04215,25.81755,1.37128,0.21177,0.04500,0.03865,0.00623,1.06308,0.51795,...,0.06931,7.15523,0.00326,0.60769,0.00391,0.05439,0.19628,0.16298,0.43104,0.07151
k__Bacteria|p__Proteobacteria,59.78683,14.27017,11.18887,15.32872,5.65027,98.34982,99.75865,19.08528,2.86320,71.37243,...,84.29704,66.36095,0.02253,18.85434,0.05037,46.68602,2.54412,2.49214,4.37883,2.16975


In [13]:
abundance.to_csv('../Data_preprocess/1.1. WMS_all.tsv', sep='\t')

In [17]:
Kingdom = [i for i in abundance.index if i.split('|')[-1][0]=='k']
abundance.loc[Kingdom, :].to_csv('../Data_preprocess/1.1. WMS_Kingdom.tsv', sep='\t')
Phylum = [i for i in abundance.index if i.split('|')[-1][0]=='p']
abundance.loc[Phylum, :].to_csv('../Data_preprocess/1.1. WMS_Phylum.tsv', sep='\t')
Class = [i for i in abundance.index if i.split('|')[-1][0]=='c']
abundance.loc[Class, :].to_csv('../Data_preprocess/1.1. WMS_Class.tsv', sep='\t')
Order = [i for i in abundance.index if i.split('|')[-1][0]=='o']
abundance.loc[Order, :].to_csv('../Data_preprocess/1.1. WMS_Order.tsv', sep='\t')
Family = [i for i in abundance.index if i.split('|')[-1][0]=='f']
abundance.loc[Family, :].to_csv('../Data_preprocess/1.1. WMS_Family.tsv', sep='\t')
Genus = [i for i in abundance.index if i.split('|')[-1][0]=='g']
abundance.loc[Genus, :].to_csv('../Data_preprocess/1.1. WMS_Genus.tsv', sep='\t')
Species = [i for i in abundance.index if i.split('|')[-1][0]=='s']
abundance.loc[Species, :].to_csv('../Data_preprocess/1.1. WMS_Species.tsv', sep='\t')

# Function

In [24]:
path = '../Data/WMS/'
namemap = {'1_2018_8_20':'1_26', 'B19_5':'19_5'}
files = [i for i in os.listdir(path) if 'pathabundance' in i]
abundance = pd.DataFrame()
colummns = []
for f in files:
    name = f.split('_paired')[0].strip()
    temp = pd.read_csv(path+f, sep='\t', index_col=0)
    #temp
    abundance = pd.concat([abundance, temp], axis=1, sort=False)
    colummns.append('C'+namemap.get(name, name))
abundance.columns = colummns
abundance = abundance.fillna(0.0)
abundance.head(3)

,C11_13,C11_22,C11_25,C11_26,C11_27,C11_4,C11_5,C12_13,C12_22,C12_25,...,C35_22,C35_27,C35_5,C4_26,C4_27,C5_22,C5_25,C5_27,C5_4,C19_5
# Pathway,,,,,,,,,,,,,,,,,,,,,
UNMAPPED,3.286113e+06,2.645572e+06,4.318300e+06,2.547671e+06,2.673637e+06,2.457996e+06,3.361266e+06,2.069896e+06,2.546893e+06,2.728858e+06,...,2.428257e+06,2.323319e+06,2.441449e+06,2.289273e+06,1.540263e+06,2.741991e+06,2.097654e+06,1.985867e+06,1.945917e+06,2.139670e+06
UNINTEGRATED,1.581775e+07,7.157036e+06,1.042615e+07,8.939702e+06,7.212366e+06,1.500154e+07,2.157955e+07,9.287710e+06,7.710586e+06,1.576826e+07,...,1.079563e+07,7.371947e+06,1.066207e+07,8.352931e+06,4.879242e+06,1.361346e+07,7.565719e+06,6.486911e+06,7.049065e+06,6.537105e+06
UNINTEGRATED|g__Klebsiella.s__Klebsiella_quasipneumoniae,9.335114e+06,1.172812e+06,6.991006e+05,3.731432e+05,0.000000e+00,1.222217e+07,1.718161e+07,0.000000e+00,2.763817e+04,4.583534e+05,...,5.461542e+06,4.576693e+04,0.000000e+00,1.278715e+06,0.000000e+00,1.235537e+06,6.091902e+04,1.470610e+04,0.000000e+00,8.869717e+03


In [32]:
pathway = [i for i in abundance.index if '|' not in i and i not in ['UNMAPPED', 'UNINTEGRATED']]
pathway_abundance = abundance.loc[pathway, :]
pathway_abundance.index = [i.split(':')[0].strip() for i in pathway_abundance.index]
pathway_abundance = pathway_abundance.fillna(0.0)
pathway_abundance.shape
pathway_abundance.to_csv('../Data_preprocess/1.1. WMS_Pathway.tsv', sep='\t')

(499, 66)